In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import utils

In [2]:
patent_data = utils.load_patent_data('../data/patsnap_data.xlsx','sheet1')

In [3]:
patent_data.head(3)

,Number,Publication Number,Title,Legal Status & Events,Current Assignee,Application Date,IPC,Patent Valuation,Abstract,Abstract_English,Claims,Title_English,CPC
0,1,US6056237A,Sonotube compatible unmanned aerial vehicle an...,Non-payment,1281329 ALBERTA LTD.,1997-06-25,B64D1/02 | B64D1/00 | B64D33/02 | B64C39/00 | ...,-,The present invention is generally comprised o...,The present invention is generally comprised o...,I claim:_x000D_\n1. A sonotube compatible unma...,Sonotube compatible unmanned aerial vehicle an...,B64C3/40 | B64C5/12 | B64C39/024 | B64D1/02 | ...
1,2,US8511606B1,Unmanned aerial vehicle base station,Granted,THE BOEING COMPANY,2009-12-09,B64D41/00,"$ 56,000","A method and apparatus comprising a platform, ...","A method and apparatus comprising a platform, ...",1. An apparatus comprising:_x000D_\na platform...,Unmanned aerial vehicle base station,B64C39/028 | B64C39/024 | B64C2201/066 | B64C2...
2,3,US8948935B1,Providing a medical support device via an unma...,Granted | Transfer,WING AVIATION LLC,2013-01-02,G06Q10/00 | B64C39/02 | G16H40/67,"$ 79,000",Embodiments described herein may relate to an ...,Embodiments described herein may relate to an ...,1. An unmanned aerial vehicle (UAV) comprising...,Providing a medical support device via an unma...,A61B5/00 | A61B19/0264 | B64C39/024 | G06F19/3...


In [4]:
# Remove duplicates
patent_data = patent_data.drop_duplicates(subset=['Title_English', 'Abstract_English'])

In [5]:
# Reset index
patent_data = patent_data.reset_index(drop=True)

In [6]:
patent_data

,Number,Publication Number,Title,Legal Status & Events,Current Assignee,Application Date,IPC,Patent Valuation,Abstract,Abstract_English,Claims,Title_English,CPC
0,1,US6056237A,Sonotube compatible unmanned aerial vehicle an...,Non-payment,1281329 ALBERTA LTD.,1997-06-25,B64D1/02 | B64D1/00 | B64D33/02 | B64C39/00 | ...,-,The present invention is generally comprised o...,The present invention is generally comprised o...,I claim:_x000D_\n1. A sonotube compatible unma...,Sonotube compatible unmanned aerial vehicle an...,B64C3/40 | B64C5/12 | B64C39/024 | B64D1/02 | ...
1,2,US8511606B1,Unmanned aerial vehicle base station,Granted,THE BOEING COMPANY,2009-12-09,B64D41/00,"$ 56,000","A method and apparatus comprising a platform, ...","A method and apparatus comprising a platform, ...",1. An apparatus comprising:_x000D_\na platform...,Unmanned aerial vehicle base station,B64C39/028 | B64C39/024 | B64C2201/066 | B64C2...
2,3,US8948935B1,Providing a medical support device via an unma...,Granted | Transfer,WING AVIATION LLC,2013-01-02,G06Q10/00 | B64C39/02 | G16H40/67,"$ 79,000",Embodiments described herein may relate to an ...,Embodiments described herein may relate to an ...,1. An unmanned aerial vehicle (UAV) comprising...,Providing a medical support device via an unma...,A61B5/00 | A61B19/0264 | B64C39/024 | G06F19/3...
3,4,US20100250022A1,Useful unmanned aerial vehicle,Withdrawn-Deemed,"AIR RECON, INC.",2006-12-29,G05D1/00 | B64C13/20 | G06F3/048,-,An unmanned aerial vehicle (UAV) addresses rem...,An unmanned aerial vehicle (UAV) addresses rem...,1-17. (canceled)_x000D_\n18. A method of opera...,Useful unmanned aerial vehicle,B64C2201/141 | B64C2201/145 | G05D1/0094 | G05...
4,5,US20110084162A1,Autonomous Payload Parsing Management System a...,Abandoned-Undetermined,HONEYWELL INTERNATIONAL INC.,2009-10-09,B64C29/00 | G01M1/12 | B64C17/10 | B64D37/14 |...,-,An unmanned aerial vehicle (UAV) for making pa...,An unmanned aerial vehicle (UAV) for making pa...,1. An unmanned aerial vehicle (UAV) for making...,Autonomous Payload Parsing Management System a...,B64C39/024 | B64C2201/027 | B64C2201/088 | B64...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,9996,CN206407426U,Take precautions against earthquakes and swing...,Granted,华南智能机器人创新研究院,2016-12-30,B65D90/52 | B64C39/02,"$ 3,600",The utility model discloses a taking precautio...,The utility model discloses a taking precautio...,1.一种防震荡药箱，其特征在于，包括外壳、用来密封外壳的上盖、用于防震荡的药液分隔结构；外壳...,Take precautions against earthquakes and swing...,-
9943,9997,CN106708070A,Aerial photographing control method and apparatus,Granted,深圳市道通智能航空技术股份有限公司,2015-08-17,G05D1/08,"$ 130,000","The invention, which relates to the technical ...","The invention, which relates to the technical ...",1.一种航拍控制方法，其特征在于，包括步骤：_x000D_\n预设无人机状态数据与触发无人机...,Aerial photographing control method and apparatus,-
9944,9998,CN205864058U,A redundancy power supply for unmanned aerial ...,Non-payment,深圳光启空间技术有限公司,2016-07-14,H02J9/06,-,The utility model discloses a redundancy power...,The utility model discloses a redundancy power...,1.一种用于无人机系统的冗余电源，其特征在于，包括：_x000D_\n电源保护模块，与电源模...,A redundancy power supply for unmanned aerial ...,-
9945,9999,GB2377683A,Composite of unmanned aerial vehicles,Withdrawn-Undetermined,BAE SYSTEMS PLC,2001-07-20,B64D7/08 | B64D5/00 | B64C39/02 | F42B15/36 | ...,-,The aerial vehicle comprises a plurality of UA...,The aerial vehicle comprises a plurality of UA...,Claims_x000D_\n1 An aerial vehicle comprising ...,Composite of unmanned aerial vehicles,B64C39/024 | B64C2201/082 | B64C2201/102 | B64...


In [7]:
# Extract relevant features (Title and Abstract)
titles = patent_data['Title_English'].astype(str)
abstracts = patent_data['Abstract_English'].astype(str)

In [8]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [9]:
# Embed patent titles and abstracts
title_embeddings = model.encode(titles)
abstract_embeddings = model.encode(abstracts)

In [10]:
# Normalize embeddings
title_embeddings = title_embeddings / np.linalg.norm(title_embeddings, axis=1, keepdims=True)
abstract_embeddings = abstract_embeddings / np.linalg.norm(abstract_embeddings, axis=1, keepdims=True)

In [11]:
# Average embeddings
document_embeddings = (title_embeddings + abstract_embeddings) / 2

In [12]:
# Define a function for semantic search
def semantic_search(query, top_k=5):
    # Embed the query
    query_embedding = model.encode([query])
    query_embedding /= np.linalg.norm(query_embedding)
    
    # Compute cosine similarities between query embedding and document embeddings
    similarities = cosine_similarity(query_embedding, document_embeddings)[0]
    
    # Sort document indices based on similarity scores
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    return top_indices, similarities[top_indices]

In [13]:
# Example usage
query = "UAV Model aerospace flight"
top_k = 5
document_indices, scores = semantic_search(query, top_k=top_k)

# Retrieve relevant patent data
relevant_patents = patent_data.iloc[document_indices]

In [14]:
# Display results
print("Relevant Patents:")
for i in range(top_k):
    print(f"Patent ID: {relevant_patents.index[i]}, Score: {scores[i]}")
    print(f"Title: {relevant_patents.iloc[i]['Title_English']}")
    print(f"Abstract: {relevant_patents.iloc[i]['Abstract_English']}")
    print()

Relevant Patents:
Patent ID: 581, Score: 0.7572813034057617
Title: Transitioning an unmanned aerial vehicle to horizontal flight
Abstract: This disclosure describes a configuration of an unmanned aerial vehicle (UAV) that will facilitate extended flight duration. The UAV may have any number of lifting motors. For example, the UAV may include four lifting motors (also known as a quad-copter), eight lifting motors (octo-copter), etc. Likewise, to improve the efficiency of horizontal flight, the UAV also includes a thrusting motor and propeller assembly that is oriented at approximately ninety degrees to one or more of the lifting motors. When the UAV is moving horizontally, it may be determined if the horizontal airspeed of the UAV exceeds an airspeed threshold. If the horizontal airspeed exceeds the airspeed threshold, the thrusting motor may be engaged and the thrusting propeller will aid in the horizontal propulsion of the UAV.

Patent ID: 5060, Score: 0.752551257610321
Title: Unmanne

In [16]:
# Define a function to filter patents based on relevancy scores
def filter_patents(document_indices, scores, threshold):
    relevant_patents_indices = [idx for idx, score in zip(document_indices, scores) if score > threshold]
    return relevant_patents_indices

# Example usage
query = "Artificial intelligence in healthcare"
top_k = 5
threshold = 0.5  # Adjust the threshold as needed
document_indices, scores = semantic_search(query, top_k=top_k)

# Filter patents based on relevancy scores
relevant_patents_indices = filter_patents(document_indices, scores, threshold)

# Retrieve relevant patent data
relevant_patents = patent_data.iloc[relevant_patents_indices]

# Display filtered results
print("Relevant Patents:")
for i, idx in enumerate(relevant_patents_indices):
    print(f"Patent ID: {idx}, Score: {scores[i]}")
    print(f"Title: {relevant_patents.iloc[i]['Title_English']}")
    print(f"Abstract: {relevant_patents.iloc[i]['Abstract_English']}")
    print()

Relevant Patents:
Patent ID: 8955, Score: 0.5781064033508301
Title: Health and guarantee integrated management universal system of unmanned intelligent equipment
Abstract: The invention discloses a health and guarantee integrated management universal system of unmanned intelligent equipment, and belongs to the technical field of unmanned intelligent equipment. The health and guarantee integrated management universal system of the unmanned aerial vehicle intelligent equipment comprises a data server host and a WEB display terminal, wherein the data server host is connected with the WEB display terminal by a WEB interface, a data detection unit and a data transmission unit are arranged on unmanned intelligent equipment manufacturing matched equipment, the data server host is in signal connection with the data detection unit and the data transmission unit, the data server host comprises the following units of an external data receiving unit, and the external data receiving unit is used fo